In [8]:
PORTAL_PORT=49051#порт Портала СРП
PORTAL_HOST='0.0.0.0'#хост портала
DATABASE_PORT=27017#порт базы данных

import datetime
from bson.objectid import ObjectId
import random
import string
import hmac
import hashlib
from hashlib import sha256
from flask import Flask
from flask import request,redirect
from flask import Response
from flask import render_template
from os import listdir
from os.path import isfile, join
import os
from pymongo import MongoClient
import pymongo
import os.path

if os.path.isfile('.ADMIN_PASSWORD'):#проверка наличия файла с паролем администратора 
    with open('.ADMIN_PASSWORD','r') as f:#если файл есть, пароль считывается из него
        ADMIN_PASSWORD = f.read()
else:
    with open('.ADMIN_PASSWORD','w') as f:#если его нет, он создаётся со стандартным значением пароля
        ADMIN_PASSWORD = "ADMIN_PASSWORD"
        f.write(ADMIN_PASSWORD)


app = Flask(__name__)#создание объекта сервера
app.config['TEMPLATES_AUTO_RELOAD'] = True

@app.after_request#запрет кеширования
def add_header(r):
    r.headers["Cache-Control"] = "no-cache, no-store, must-revalidate"
    r.headers["Pragma"] = "no-cache"
    r.headers["Expires"] = "0"
    r.headers['Cache-Control'] = 'public, max-age=0'
    return r

@app.route('/create')#страница с формой создания
def create():
    return render_template("formCreate.html")

@app.route('/createError')#страница с ошибкой создания
def createError():
    return render_template("createError.html")

@app.route('/saveError')#страница с ошибкой сохранения
def saveError():
    return render_template("saveError.html")

@app.route("/createSave",methods=['POST'])#запрос на создание приложения
def createSave():
    if request.form.get('name', "")=="":#проверка заполнения полей
        return redirect("/createError")
    if request.form.get('email', "")=="":
        return redirect("/createError")
    if request.form.get('devName', "")=="":
        return redirect("/createError")
    if request.form.get('appSite', "")=="":
        return redirect("/createError")
    if len(request.form.get('appPassword', ""))<10:
        return redirect("/createError")
    form_text=["name","desc","email","devName","appSite"]
    form_checks=["list","read","write","delete","mkdir","rename"]
    appObject={i:request.form.get(i, "") for i in form_text}
    for i in form_checks:
        t=request.form.get(i, "off")
        if t=="on":
            t="zapros"#если при заполнении формы запрошено полномочие, оно заменяется на запрашиваемое
        appObject[i]=t
    appObject["salt"]=''.join(random.choices(string.ascii_letters + string.digits, k=10))#генерация соли для пароли
    #генераиця хеша пароля с солью
    appObject["passwordHash"]=sha256((appObject["salt"]+request.form.get('appPassword', "")).encode('utf-8')).hexdigest()
    #генерация секретного ключа приложения
    appObject["secretKey"]=sha256(''.join(random.choices(string.ascii_letters + string.digits, k=10)).encode('utf-8')).hexdigest()
    appObject["date"]=datetime.datetime.utcnow()#запись времени создания
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    result=db.apps.insert_one(appObject)#добавление приложения
    #добавление временной ссылки для отображения секретного ключа 1 раз
    link=db.links.insert_one({"appID":result.inserted_id})
    return redirect(f"/secretKey?link_id={link.inserted_id}")#переадресация для отображения секретного ключа

@app.route("/getApp",methods=['GET'])#страница с карточкой приложения
def getApp():
    appID=request.args.get('id',"Ошибка")
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    appObject=db.apps.find_one({"_id":ObjectId(appID)})
    return render_template("getApp.html",appObject=appObject)

@app.route("/",methods=['GET'])#отображение списка приложений
def appList():
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    apps=list()
    for i in db.apps.find({}).sort("date",pymongo.DESCENDING):#сортировка приложений от новых к старым
        t=i
        m=["list","read","write","delete"]
        status="off"
        for i2 in m:
            if i[i2] == "on":
                status="on"
        for i2 in m:
            if i[i2] == "zapros":
                status="zapros"
        t["status"]=status
        apps.append(t)
    return render_template("appList.html",apps=apps,mode="read")
    
@app.route("/admin",methods=['GET'])#список приложений для администратора
def admin():
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    apps=list()
    for i in db.apps.find({}).sort("date",pymongo.DESCENDING):
        t=i
        m=["list","read","write","delete"]
        status="off"
        for i2 in m:
            if i[i2] == "on":
                status="on"
        for i2 in m:
            if i[i2] == "zapros":
                status="zapros"
        t["status"]=status
        apps.append(t)
    return render_template("appList.html",apps=apps,mode="admin")

@app.route("/adminApp")#редактирование карточки приложения
def adminApp():
    appID=request.args.get('id',"Ошибка")
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    appObject=db.apps.find_one({"_id":ObjectId(appID)})
    return render_template("adminApp.html",appObject=appObject)

@app.route("/adminSave",methods=['POST'])#сохранение карточки приложения после редактирования
def adminSave():
    if request.form.get('name', "")=="":#проверка корректности заполнения формы
        return redirect("/saveError")
    if request.form.get('email', "")=="":
        return redirect("/saveError")
    if request.form.get('devName', "")=="":
        return redirect("/saveError")
    if request.form.get('appSite', "")=="":
        return redirect("/saveError")
    if request.form.get("adminPassword","") != ADMIN_PASSWORD:#проверка пароля администратора
        return redirect("/saveError")
    appID=request.form.get('appID', "")
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    db.apps.update_one({"_id":ObjectId(appID)},#изменение объекта в БД
                       {'$set': 
                       {
                           "name":request.form.get('name', ""),
                           "desc":request.form.get('appDesc', ""),
                           "devName":request.form.get('devName', ""),
                           "email":request.form.get('email', ""),
                           "list":request.form.get('list', ""),
                           "read":request.form.get('read', ""),
                           "write":request.form.get('write', ""),
                           "delete":request.form.get('delete', ""),
                           "mkdir":request.form.get('mkdir', ""),
                           "rename":request.form.get('rename', ""),
                           "appSite":request.form.get('appSite', "")
                       }})
    return redirect(f"/adminApp?id={appID}")

@app.route("/adminDelete",methods=['POST'])#запрос на удаление приложения
def adminDelete():
    if request.form.get("adminPassword","") != ADMIN_PASSWORD:
        return redirect("/saveError")
    appID=request.form.get('appID', "")
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    db.apps.delete_one({"_id":ObjectId(appID)})
    return redirect("/admin")

@app.route('/secretKey')#запрос на показ секретного ключа
def secretKey():
    linkID=request.args.get('link_id',"Ошибка")
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    linkObject=db.links.find_one({"_id":ObjectId(linkID)})
    appObject=db.apps.find_one({"_id":ObjectId(linkObject["appID"])})
    db.links.delete_one({"_id":ObjectId(linkID)})
    return render_template("secretKey.html",appObject=appObject)

@app.route("/devApp")#управление приложением для разработчика
def devApp():
    appID=request.args.get('id',"Ошибка")
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    appObject=db.apps.find_one({"_id":ObjectId(appID)})
    return render_template("devApp.html",appObject=appObject)

@app.route("/devNewKey",methods=["POST"])#запрос разработчика на генерацию нового ключа
def devNewKey():
    appID=request.form.get('appID', "")
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    appObject=db.apps.find_one({"_id":ObjectId(appID)})
    salt=appObject["salt"]
    dbHash=appObject["passwordHash"]
    #вычисление хеша из соли и введённого пароля
    formHash=sha256((salt+request.form.get('devPassword', "")).encode('utf-8')).hexdigest()
    if dbHash != formHash:#сравнение хеша из БД и полученного. Если совпадают - пароль верный
        return redirect("/saveError")#если нет - страница ошибки
    newKey=sha256(''.join(random.choices(string.ascii_letters + string.digits, k=10)).encode('utf-8')).hexdigest()#новый ключ
    db.apps.update_one({"_id":ObjectId(appID)},#изменение ключа в БД
                       {'$set': 
                       {
                           "secretKey":newKey
                       }})
    link=db.links.insert_one({"appID":ObjectId(appObject["_id"])})#создание временной ссылки
    return redirect(f"/secretKey?link_id={link.inserted_id}")#и переадресация на неё для показа секретного ключа

@app.route("/devDelete",methods=["POST"])#запрос разработчика на удаление приложения
def devDelete():
    appID=request.form.get('appID', "")
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    appObject=db.apps.find_one({"_id":ObjectId(appID)})
    salt=appObject["salt"]
    dbHash=appObject["passwordHash"]
    formHash=sha256((salt+request.form.get('devPassword', "")).encode('utf-8')).hexdigest()
    if dbHash != formHash:
        return redirect("/saveError")
    newKey=sha256(''.join(random.choices(string.ascii_letters + string.digits, k=10)).encode('utf-8')).hexdigest()
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    db.apps.delete_one({"_id":ObjectId(appID)})#удаление приложения
    return redirect("/")

@app.route("/devChangePassword",methods=["POST"])#запрос разработчика на изменение пароля приложения
def devChangePassword():
    if len(request.form.get('newPassword', ""))<10:
        return redirect("/saveError")
    appID=request.form.get('appID', "")
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    appObject=db.apps.find_one({"_id":ObjectId(appID)})
    salt=appObject["salt"]
    dbHash=appObject["passwordHash"]
    formHash=sha256((salt+request.form.get('devPassword', "")).encode('utf-8')).hexdigest()
    if dbHash != formHash:
        return redirect("/saveError")
    salt=''.join(random.choices(string.ascii_letters + string.digits, k=10))
    passwordHash=sha256((salt+request.form.get('newPassword', "")).encode('utf-8')).hexdigest()
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    db.apps.update_one({"_id":ObjectId(appID)},#изменение пароля
                       {'$set': 
                       {
                           "passwordHash":passwordHash,
                           "salt":salt,
                       }})
    return redirect(f"/getApp?id={appID}")

@app.route("/adminChangePassword",methods=["POST"])#запрос администратора на изменение пароля приложения
def adminChangePassword():
    if len(request.form.get('newPassword', ""))<10:
        return redirect("/saveError")
    appID=request.form.get('appID', "")
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    appObject=db.apps.find_one({"_id":ObjectId(appID)})
    if request.form.get("adminPassword","") != ADMIN_PASSWORD:
        return redirect("/saveError")
    salt=''.join(random.choices(string.ascii_letters + string.digits, k=10))
    passwordHash=sha256((salt+request.form.get('newPassword', "")).encode('utf-8')).hexdigest()
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    db.apps.update_one({"_id":ObjectId(appID)},#изменение пароля
                       {'$set': 
                       {
                           "passwordHash":passwordHash,
                           "salt":salt,
                       }})
    return redirect(f"/adminApp?id={appID}")
                        
                        
app.run(host=PORTAL_HOST, port=PORTAL_PORT, threaded=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:49051/ (Press CTRL+C to quit)
